# <font color='darkorange'> **Fine-tuning XLM-RoBERTa for extractive Question Answering in Catalan** </font>

In this notebook we will fine-tune XLM-RoBERTa for extractive Question Answering in Catalan using Hugging Face transformers.

In [ ]:
# Install Hugging Face transformers and datasets.
!pip install transformers datasets huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 69.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 30.1 MB/s eta 0:00:00


In [ ]:
import transformers
# Before continuing, make sure version of transformers is at least 4.16. 
print(transformers.__version__)

4.27.2


In [ ]:
model_checkpoint = "xlm-roberta-base"

## **1. Loading Viquiquad**

In [ ]:
# Use datasets library to load the dataset and get the metric we need for evaluation.
from datasets import load_dataset, load_metric

In [ ]:
# Connect to Google Drive to load the dataset.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load Viquiquad from Google Drive
#   (originally downloaded from https://huggingface.co/datasets/projecte-aina/viquiquad/tree/main).
path = "/content/drive/My Drive/viquiquad"

script = path+"/viquiquad.py"
dataset = {"train": path+"/train.json",
           "dev": path+"/dev.json",
           "test": path+"/test.json"}

raw_dataset = load_dataset(script, data_files=dataset)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset viquiquad downloaded and prepared to /root/.cache/huggingface/datasets/viquiquad/default-5315f57053f38ae0/1.0.1/bd00175ebc7354f52ec9b50720441e2b918cc8a93efecb056e0231e07e378717. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11259
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 1493
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 1428
    })
})

## **2. Preprocessing the dataset**



In [ ]:
# Instantiate tokenizer with the AutoTokenizer.from_pretrained method 
#   to get the tokenizer corresponding to the model architecture and 
#   to download the vocabulary used when pretraining it.
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
# The tokenizer will tokenize the inputs and convert the tokens to their corresponding IDs in the pretrained vocabulary.

In [ ]:
max_length = 384  # Maximum length of a feature (question and context).
# Examples longer than max_length will be splitted into several input features.
# The context of each of these features will overlap a bit the context of the previous feature
#   (just in case the answer lies at the point where the context is splitted).
doc_stride = 128  # Allowed overlap between the features when splitting is performed.

In [ ]:
def prepare_train_features(examples):
    # Tokenize examples with truncation and padding, but keep the overflows using a stride.
    
    tokenized_examples = tokenizer(examples["question"],
                                   examples["context"],
                                   truncation="only_second", # Only truncate context (not question).
                                   max_length=max_length,
                                   stride=doc_stride, 
                                   return_overflowing_tokens=True,
                                   return_offsets_mapping=True, # Map to find start and end positions of the answers in the tokens.
                                   padding="max_length")
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping") # Map from a feature to its corresponding example.
    offset_mapping = tokenized_examples.pop("offset_mapping") # Map from token to character position in the original context.
    
    # Label examples:
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i]
        # Grab the sequence corresponding to that example (to know what is the context and what is the question):
        #   returns None for special tokens, 
        #   and 0 or 1 depending on whether the corresponding token comes from the first sentence (question) or the second (context).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # Index of the example containing this span of text (as one example can give several spans).
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        start_char = answers[0]["answer_start"]
        end_char = answers[0]["answer_start"] + len(answers[0]["text"])

        # Start token index of the current span in the text.
        token_start_index = 0
        while sequence_ids[token_start_index] != 1:
          token_start_index += 1

        # End token index of the current span in the text.
        token_end_index = len(input_ids) - 1
        while sequence_ids[token_end_index] != 1:
          token_end_index -= 1

        # Detect if the answer is out of the span. If so, the label is (0,0).
        if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(0)
                tokenized_examples["end_positions"].append(0)
        # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
        else:
            while (token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char):
                token_start_index += 1
            tokenized_examples["start_positions"].append(token_start_index - 1)
            while offsets[token_end_index][1] >= end_char:
                token_end_index -= 1
            tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [ ]:
# Apply function to dataset.
tokenized_dataset = raw_dataset.map(prepare_train_features, 
                                    batched=True, 
                                    remove_columns=raw_dataset["train"].column_names)

## **3. Fine-tuning the model**

In [ ]:
# Download pretrained model.
# Since our task is QA, we use the TFAutoModelForQuestionAnswering class. 
from transformers import TFAutoModelForQuestionAnswering
# Like with the tokenizer, from_pretrained method will download and cache the model.
model = TFAutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

All model checkpoint layers were used when initializing TFXLMRobertaForQuestionAnswering.

Some layers of TFXLMRobertaForQuestionAnswering were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['qa_outputs']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
batch_size = 17      # Set to 32 in Armengol-Estapé et al.
learning_rate = 5e-5 
num_train_epochs = 5 # Set to 10 in Armengol-Estapé et al.

In [ ]:
# Convert train and validation sets to tf.data.Dataset.
train_set = model.prepare_tf_dataset(tokenized_dataset["train"],
                                     shuffle=True,
                                     batch_size=batch_size)
dev_set = model.prepare_tf_dataset(tokenized_dataset["validation"],
                                   shuffle=False,
                                   batch_size=batch_size)

In [ ]:
# Create optimizer and specify loss function.
from transformers import create_optimizer # AdamW optimizer.
total_train_steps = len(train_set)*num_train_epochs # Compute total number of training steps.
optimizer, schedule = create_optimizer(init_lr=learning_rate, 
                                       num_warmup_steps=0, 
                                       num_train_steps=total_train_steps)

In [ ]:
# Compile the model.
import tensorflow as tf
model.compile(optimizer=optimizer, 
              jit_compile=True, 
              metrics=["accuracy"])

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [ ]:
# Fine-tune the model.
model.fit(train_set,
          validation_data=dev_set,
          epochs=num_train_epochs)

Epoch 1/5
698/698 [==============================] - 1010s 1s/step - loss: 1.3143 - end_logits_accuracy: 0.6868 - start_logits_accuracy: 0.6176 - val_loss: 0.7495 - val_end_logits_accuracy: 0.7976 - val_start_logits_accuracy: 0.7583
Epoch 2/5
698/698 [==============================] - 914s 1s/step - loss: 0.5724 - end_logits_accuracy: 0.8517 - start_logits_accuracy: 0.7986 - val_loss: 0.6324 - val_end_logits_accuracy: 0.8320 - val_start_logits_accuracy: 0.8070
Epoch 3/5
698/698 [==============================] - 916s 1s/step - loss: 0.3527 - end_logits_accuracy: 0.9063 - start_logits_accuracy: 0.8658 - val_loss: 0.6291 - val_end_logits_accuracy: 0.8413 - val_start_logits_accuracy: 0.8101
Epoch 4/5
698/698 [==============================] - 915s 1s/step - loss: 0.2228 - end_logits_accuracy: 0.9409 - start_logits_accuracy: 0.9072 - val_loss: 0.7291 - val_end_logits_accuracy: 0.8364 - val_start_logits_accuracy: 0.8264
Epoch 5/5
698/698 [==============================] - 922s 1s/step - los

## **4. Evaluation**

#### **4.1. Preprocessing the test set**

Answers are classified using the score obtained by adding the start and end logits. The best indices in the start and end logits are selected (according to the hyperparameter `n_best_size`). The answers this predicts are then checked one by one and sorted by their score to keep the best one. 

To check if a given span is inside the context (and not the question) and to get back the text inside, in the test features we keep (1) the ID of the example that generated the feature (as one example can generate several features), and (2) the offset mapping (to map from token indices to character positions in the context). For this reason, the test set is preprocessed with the function `prepare_test_features`.


In [ ]:
def prepare_test_features(examples):
    # Tokenize examples with truncation and maybe padding, but keep the overflows using a stride.

    tokenized_examples = tokenizer(examples["question"],
                                   examples["context"],
                                   truncation="only_second", # Only truncate context (not question).
                                   max_length=max_length,
                                   stride=doc_stride,
                                   return_overflowing_tokens=True, 
                                   return_offsets_mapping=True, # Map to find start and end positions of the answers in the tokens.
                                   padding="max_length")
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping") # Map from a feature to its corresponding example.
    
    tokenized_examples["example_id"] = [] # Keep the example_id of the feature and store the offset mappings.

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1

        # Index of the example containing this span of text (as one example can give several spans).
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context 
        #   to determine if a token position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [(o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])]

    return tokenized_examples

In [ ]:
# Apply function to test set. 
test_features = raw_dataset["test"].map(prepare_test_features,
                                        batched=True,
                                        remove_columns=raw_dataset["test"].column_names)

In [ ]:
# Convert test set into a tf.data.Dataset.
test_set = model.prepare_tf_dataset(test_features,
                                    shuffle=False,
                                    batch_size=batch_size)

#### **4.2. Making predictions and processing them**

In [ ]:
# Predictions for all features.
raw_predictions = model.predict(test_set)

87/87 [==============================] - 32s 365ms/step


In [ ]:
import numpy as np
import collections
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples,
                               features,
                               all_start_logits,
                               all_end_logits,
                               n_best_size=20, # Best indices in start and end logits.
                               max_answer_length=30, # Eliminate longer answers.
                               ):
    
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    predictions = collections.OrderedDict()

    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Loop over all examples.
    for example_index, example in enumerate(tqdm(examples)):
        feature_indices = features_per_example[example_index] # Indices of the features associated to the current example.
        min_null_score = None
        valid_answers = []
        context = example["context"]
        
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # Grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # To be able to map some the positions in our logits to span of texts in the original context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Out-of-scope answers are not considered, 
                    #   either because the indices are out of bounds 
                    #   or because they correspond to part of the input_ids that are not in the context.
                    if (start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or not offset_mapping[start_index]
                        or not offset_mapping[end_index]):
                        continue
                    # Answers with a length that is either < 0 or > max_answer_length are not considered.
                    if (end_index < start_index
                        or end_index - start_index + 1 > max_answer_length):
                        continue
                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {"score": start_logits[start_index] + end_logits[end_index],
                         "text": context[start_char:end_char]})

        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # Fake prediction to avoid failure if there is not a single non-null prediction.
            best_answer = {"text": "", "score": 0.0}

        # Select final answer: the best one or the null answer.
        answer = (best_answer["text"] if best_answer["score"] > min_null_score else "")
        predictions[example["id"]] = answer

    return predictions

In [ ]:
# Apply post-processing function to raw predictions. 
final_predictions = postprocess_qa_predictions(raw_dataset["test"],
                                               test_features,
                                               raw_predictions["start_logits"],
                                               raw_predictions["end_logits"])

Post-processing 1428 example predictions split into 1470 features.


  0%|          | 0/1428 [00:00<?, ?it/s]

#### **4.3. Computing the metrics**

In [ ]:
# Load metric from the datasets library.
metric = load_metric("squad_v2")

In [ ]:
# Format predictions and labels as a list of dictionaries.
formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0}
                         for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} 
              for ex in raw_dataset["test"]]

In [ ]:
# Compute the metric.
metric.compute(predictions=formatted_predictions, 
               references=references)

{'exact': 73.03921568627452,
 'f1': 87.53939735657931,
 'total': 1428,
 'HasAns_exact': 73.03921568627452,
 'HasAns_f1': 87.53939735657931,
 'HasAns_total': 1428,
 'best_exact': 73.03921568627452,
 'best_exact_thresh': 0.0,
 'best_f1': 87.53939735657931,
 'best_f1_thresh': 0.0}